In [28]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from sklearn.impute import SimpleImputer


data = pd.read_csv(r"C:\Users\Administrator\Desktop\机器学习\幸存者.csv"
                   ,index_col=0
                   ,encoding='gb18030')

Age = data.loc[:,"Age"].values.reshape(-1,1)
imp_median = SimpleImputer(strategy="median")           #用中位数填补
imp_median = imp_median.fit_transform(Age)
#在这里我们使用中位数填补Age
data.loc[:,"Age"] = imp_median
#使用众数填补Sex，因为这一属性非数字是文字！
Sex = data.loc[:,"Sex"].values.reshape(-1,1)
imp_mode = SimpleImputer(strategy = "most_frequent")
data.loc[:,"Sex"] = imp_mode.fit_transform(Sex)
 
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Age     10 non-null     float64
 1   Sex     10 non-null     object 
 2   Sur     10 non-null     object 
dtypes: float64(1), object(2)
memory usage: 320.0+ bytes


In [29]:
 #接口categories_对应LabelEncoder的接口classes_，一模一样的功能
data_ = data.copy()
 
data_
 

,Age,Sex,Sur
ID,,,
0,3.0,M,Y
1,21.0,F,N
2,43.0,F,N
3,32.0,M,N
4,12.0,M,N
5,35.0,M,N
6,32.0,M,N
7,45.0,F,Y
8,22.0,F,N


In [32]:

from sklearn.preprocessing import OneHotEncoder
X = data.iloc[:,1:-1]
#选取原带有文字的数据 

In [33]:
enc = OneHotEncoder(categories='auto').fit(X)#实例化：独热中的规定：categories='auto'判断一个特征有几列。
result = enc.transform(X).toarray()#将转变为数组！！！

In [34]:
result

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.]])

### 变成了两列：这是因为特征中只有性别是文字，性别中有两种结果，所以生成为2列。假设还有一个文字特征（喜好）为：踢球，游泳，跳绳。那么最终生成3+2=5列。

In [35]:
#依然可以直接一步到位，但为了给大家展示模型属性，所以还是写成了三步
OneHotEncoder(categories='auto').fit_transform(X).toarray()
 

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.]])

In [36]:
#依然可以还原
pd.DataFrame(enc.inverse_transform(result))
 

,0
0,M
1,F
2,F
3,M
4,M
5,M
6,M
7,F
8,F
9,M


In [37]:
enc.get_feature_names()#这一行的作用就是显示哑变量的列标题！

array(['x0_F', 'x0_M'], dtype=object)

In [38]:
result
result.shape

(10, 2)

In [40]:
#axis=1,表示跨行进行合并，也就是将两表左右相连，如果是axis=0，就是将量表上下相连
newdata = pd.concat([data,pd.DataFrame(result)],axis=1)
'''将哑变量矩阵resualt和原始data连接。'''
 

'将哑变量矩阵resualt和原始data连接。'

In [41]:
newdata.head()
#这里就是将哑变量矩阵resualt和原始data连接之后的表格，右边两列：参考enc.get_feature_names()
#生成的结果，0代表女F，1代表M男。

,Age,Sex,Sur,0,1
0,3.0,M,Y,0.0,1.0
1,21.0,F,N,1.0,0.0
2,43.0,F,N,1.0,0.0
3,32.0,M,N,0.0,1.0
4,12.0,M,N,0.0,1.0


In [43]:
newdata.drop(["Sex"],axis=1,inplace=True)#删除之前的Sex，没用了！

In [44]:
newdata.columns = ["Age","Survived","Female","Male"]#列标题重命名。
 
newdata.head()

,Age,Survived,Female,Male
0,3.0,Y,0.0,1.0
1,21.0,N,1.0,0.0
2,43.0,N,1.0,0.0
3,32.0,N,0.0,1.0
4,12.0,N,0.0,1.0


### 二值化和分段

In [46]:
#将年龄二值化
 
data_2 = data.copy()
 
from sklearn.preprocessing import Binarizer
#第0列为年龄。
X = data_2.iloc[:,0].values.reshape(-1,1) #类为特征专用，所以不能使用一维数组

#30为界线
transformer = Binarizer(threshold=30).fit_transform(X)
 
transformer

array([[0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.]])

### 之后的操作和独热一样！！！！transformer就相当于之前的resualt。